Data source:
- https://www.insee.fr/fr/statistiques/3568638?sommaire=3568656
- https://public.opendatasoft.com/explore/dataset/buildingref-france-bpe-all-geolocated/export/?disjunctive.geocode_quality&disjunctive.equipment_name&disjunctive.category&disjunctive.reg_name&disjunctive.dep_name&disjunctive.epci_name&disjunctive.com_arm_name&disjunctive.com_arm_area_code&sort=year&refine.equipment_name=Supermarch%C3%A9&refine.equipment_name=Sup%C3%A9rette&refine.equipment_name=Hypermarch%C3%A9&location=12,48.84732,2.36361&basemap=jawg.streets

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import configparser
from urllib.parse import urlencode
import requests
import re

from shapely.geometry import Point, shape 
import geopandas as gpd
import folium
import branca.colormap as cm

In [2]:
bpe19_df = pd.read_csv('buildingref-france-bpe-all-geolocated.csv', sep=';')
bpe19_df = bpe19_df[(bpe19_df['Code Officiel Département'].isin(['974', '971', '976', '972', '973']) == False) &
                    (bpe19_df['Qualité du géocodage'] != 'Non Géolocalisée')]

In [3]:
bpe19_df['geo_point_list'] = bpe19_df['Geo Point'].apply(lambda x: x.split(','))
bpe19_df['LAT'] = bpe19_df['geo_point_list'].apply(lambda x: x[0])
bpe19_df['LAT'] = bpe19_df['LAT'].astype(float)
bpe19_df['LNG'] = bpe19_df['geo_point_list'].apply(lambda x: x[1])
bpe19_df['LNG'] = bpe19_df['LNG'].astype(float)


In [4]:
bpe_geometry = [Point(xy) for xy in zip(bpe19_df.LNG, bpe19_df.LAT)]
crs = {'init': 'epsg:4326'}
bpe19_gdf = gpd.GeoDataFrame(bpe19_df, crs=crs, geometry=bpe_geometry)

/Users/jingwen/github/python-playground/_venv/lib/python3.9/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [5]:
bpe19_gdf.head()

,Geo Point,Geo Shape,Code Officiel Région,Code Officiel Département,Code Officiel Commune / Arrondissement Municipal,Code Officiel IRIS,Année,Type,Qualité du géocodage,Description,Catégorie,Nom Officiel Région,Nom Officiel Département,Nom Officiel EPCI,Nom Officiel Commune / Arrondissement Municipal,Code Iso 3166-3 Zone,Code Officiel EPCI,geo_point_list,LAT,LNG,geometry
19,"46.157654293,6.66683646431","{""type"": ""MultiPoint"", ""coordinates"": [[6.6668...",84,74,74134,74134,2019,B201,Bonne,Supérette,Commerces,Auvergne-Rhône-Alpes,Haute-Savoie,CC du Haut-Chablais,Les Gets,FXX,247400682.0,"[46.157654293, 6.66683646431]",46.157654,6.666836,POINT (6.66684 46.15765)
20,"48.4982013079,6.89000003685","{""type"": ""MultiPoint"", ""coordinates"": [[6.8900...",44,54,54040,54040,2019,B201,Bonne,Supérette,Commerces,Grand Est,Meurthe-et-Moselle,CC de Vezouze en Piémont,Badonviller,FXX,200069433.0,"[48.4982013079, 6.89000003685]",48.498201,6.890000,POINT (6.89000 48.49820)
21,"48.8484267597,2.31358572806","{""type"": ""MultiPoint"", ""coordinates"": [[2.3135...",11,75,75107,751072702,2019,B201,Bonne,Supérette,Commerces,Île-de-France,Paris,Métropole du Grand Paris,Paris 7e Arrondissement,FXX,200054781.0,"[48.8484267597, 2.31358572806]",48.848427,2.313586,POINT (2.31359 48.84843)
22,"47.18200911,1.09556447267","{""type"": ""MultiPoint"", ""coordinates"": [[1.0955...",24,37,37111,37111,2019,B201,Bonne,Supérette,Commerces,Centre-Val de Loire,Indre-et-Loire,CC Loches Sud Touraine,Genillé,FXX,200071587.0,"[47.18200911, 1.09556447267]",47.182009,1.095564,POINT (1.09556 47.18201)
23,"46.6015795426,0.324656951694","{""type"": ""MultiPoint"", ""coordinates"": [[0.3246...",75,86,86194,861941001,2019,B201,Bonne,Supérette,Commerces,Nouvelle-Aquitaine,Vienne,CU du Grand Poitiers,Poitiers,FXX,200069854.0,"[46.6015795426, 0.324656951694]",46.601580,0.324657,POINT (0.32466 46.60158)


In [6]:
dept_geo = gpd.read_file('../geo_datasets/fr_departements.geojson', driver='GeoJSON')
commune_geo = gpd.read_file('../geo_datasets/fr_contours_communes_2019.geojson', driver='GeoJSON')

In [7]:
commune_geo = commune_geo[['insee_dep', 'insee_com', 'nom_dep', 'nom_com', 'geometry']]
commune_geo['nom_dept_com'] = commune_geo.apply(lambda row: row['nom_dep'] + ':' + row['nom_com'], axis='columns')

In [8]:
def fill_dist_dict(commune_polygon, site_point, nom_dept_com, dist_dict):
    dist = commune_polygon.boundary.distance(site_point)
    if not nom_dept_com in dist_dict:
        dist_dict[nom_dept_com] = dist
    else:
        if dist < dist_dict[nom_dept_com]:
            dist_dict[nom_dept_com] = dist
    return dist_dict


def calcul_shortest_distance(insee_dep, commune_polygon, nom_dept_com, store_dept, dist_dict, sites_gdf):
    if insee_dep in store_dept:
        dist_dict = sites_gdf[sites_gdf['Code Officiel Département'] == insee_dep].apply(
            lambda row:fill_dist_dict(commune_polygon,
                                      row['geometry'],
                                      nom_dept_com,
                                      dist_dict),
            axis='columns')
    else:
        dist_dict = sites_gdf.apply(lambda row:fill_dist_dict(commune_polygon,
                                                              row['geometry'],
                                                              nom_dept_com,
                                                              dist_dict),
                                    axis='columns')

    return dist_dict

In [9]:
%%time
bpe19_dept = bpe19_gdf['Code Officiel Département'].unique()
dist_dict = {}

dist_dict = commune_geo.apply(lambda row:calcul_shortest_distance(row['insee_dep'],
                                                                  row['geometry'],
                                                                  row['nom_dept_com'],
                                                                  bpe19_dept,
                                                                  dist_dict,
                                                                  bpe19_gdf),
                              axis='columns')

commune_geo['shortest_distance'] = commune_geo['nom_dept_com'].map(dist_dict.iloc[0, 0])

CPU times: user 17min 47s, sys: 5min 18s, total: 23min 6s
Wall time: 51min 42s


In [10]:
commune_geo.head()

,insee_dep,insee_com,nom_dep,nom_com,geometry,nom_dept_com,shortest_distance
0,971,97128,GUADELOUPE,Sainte-Anne,"POLYGON ((-61.45532 16.25808, -61.45433 16.257...",GUADELOUPE:Sainte-Anne,64.790430
1,14,14713,CALVADOS,Montillières-sur-Orne,"POLYGON ((-0.49574 49.03561, -0.49830 49.03625...",CALVADOS:Montillières-sur-Orne,0.030425
2,25,25245,DOUBS,Fontain,"POLYGON ((6.07335 47.18417, 6.07104 47.18410, ...",DOUBS:Fontain,0.019560
3,44,44105,LOIRE-ATLANTIQUE,Mouais,"POLYGON ((-1.63574 47.69615, -1.63544 47.69519...",LOIRE-ATLANTIQUE:Mouais,0.027757
4,73,73236,SAVOIE,Saint-Genix-les-Villages,"POLYGON ((5.63766 45.58963, 5.63600 45.59188, ...",SAVOIE:Saint-Genix-les-Villages,0.002305


In [ ]:
# commune_geo.to_csv('commune_geo_distance.csv', index=False)
# commune_geo = pd.read_csv('commune_geo_distance.csv')

In [11]:
commune_visu_gdf = commune_geo.copy()
commune_visu_gdf = commune_visu_gdf[['nom_dept_com', 'geometry', 'shortest_distance']]
commune_visu_gdf['shortest_distance_km'] = commune_visu_gdf['shortest_distance'] * 100
commune_visu_gdf = gpd.GeoDataFrame(commune_visu_gdf, crs=crs, geometry=commune_visu_gdf['geometry'])

commune_visu_gdf['geometry_hull'] = commune_visu_gdf.geometry.convex_hull
commune_visu_gdf = commune_visu_gdf[['nom_dept_com', 'geometry_hull', 'shortest_distance_km']]
commune_visu_gdf = gpd.GeoDataFrame(commune_visu_gdf, crs=crs, geometry=commune_visu_gdf['geometry_hull'])
commune_visu_gdf = commune_visu_gdf.drop(columns='geometry_hull')

/Users/jingwen/github/python-playground/_venv/lib/python3.9/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
<ipython-input-11-ddb3ca449220>:4: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  commune_visu_gdf = gpd.GeoDataFrame(commune_visu_gdf, crs=crs, geometry=commune_visu_gdf['geometry'])


In [12]:
# commune_visu_gdf.describe(percentiles=[0.2, 0.4, 0.6, 0.8])
commune_visu_gdf.describe(percentiles=[0.25, 0.5, 0.75, 1])


,shortest_distance_km
count,34970.000000
mean,28.179654
std,408.841644
min,0.000373
25%,1.011851
50%,2.540480
75%,4.903533
100%,7801.039084
max,7801.039084


In [13]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:22px; right: 20px; top: 20px;'>
     
<div class='legend-title'>Distances</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#7e6bc4;opacity:1;'></span>0.0 -  1.0 km</li>
    <li><span style='background:#c79ecf;opacity:1;'></span>1.0 -  2.5 km</li>
    <li><span style='background:#d6c8ff;opacity:1;'></span>2.5 -  5.0 km</li>
    <li><span style='background:#fef0ff;opacity:1;'></span>  >    5.0 km</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 12px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 10px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 20px;
    width: 40px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

In [17]:
template_title = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:0px; background-color:rgba(252, 255, 255, 0.8);
     border-radius:0px; padding: 10px; font-size:40px; color:rgb(0, 0, 0); left: 50px; top: 0px;'>
<div class='legend-title'>Distances from the municipalities to the nearest retailers</div>
<div class='legend-title'>(Hypermarket, supermarket and mini-market)</div>
</div>

</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 12px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 10px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 20px;
    width: 40px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

In [15]:
colormap_commune = cm.StepColormap(
    colors=['#7e6bc4', '#c79ecf',
            '#d6c8ff', '#fef0ff'],
    vmin=min(commune_visu_gdf['shortest_distance_km']),
    vmax=max(commune_visu_gdf['shortest_distance_km']),
    index=[0, 1, 2.5, 5, round(commune_visu_gdf['shortest_distance_km'].max(), 0)])

# '#b6c2ff', '#cec5eb', '#fad0ba', '#e5cad2'
# blue: '#7098da', '#6eb6ff', '#90f2ff', '#e0fcff'
# green: '#397d54', '#73c088', '#a8e087', '#c8ead1'

In [ ]:
colormap_commune

In [18]:
bpe_map = folium.Map(location=[46.803354, 1.8883335], zoom_start = 7)
folium.TileLayer('cartodbpositron').add_to(bpe_map)

style_function = lambda x: {
    'fillColor': colormap_commune(x['properties']['shortest_distance_km']),
    'color': '',
    'weight': 0,
    'fillOpacity': 1
}

folium.GeoJson(commune_visu_gdf,
               style_function = style_function,
               name='Commune').add_to(bpe_map)

folium.GeoJson(dept_geo,
               style_function = lambda x: {
                   'color': '#060495',
                   'weight': 1,
                   'fillOpacity': 0},
               name='Departement').add_to(bpe_map)

macro = MacroElement()
macro._template = Template(template)
macro2 = MacroElement()
macro2._template = Template(template_title)

bpe_map.get_root().add_child(macro)
bpe_map.get_root().add_child(macro2)

bpe_map.save('bpe_map.html')